In [1]:
import xgboost as xgb
import lightgbm as lgb
import sklearn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels
import statsmodels.api as sm

In [2]:
print("xgboost version: {}".format(xgb.__version__))
print("lightgbm version: {}".format(lgb.__version__))
print('sci-kit learn version: {}'.format(sklearn.__version__))
print('statsmodels version: {}'.format(statsmodels.__version__))
print('matplotlib version: {}'.format(matplotlib.__version__))

xgboost version: 0.72.1
lightgbm version: 2.1.2
sci-kit learn version: 0.19.1
statsmodels version: 0.9.0
matplotlib version: 2.2.2
